---

_You are currently looking at **version 1.2** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-data-analysis/resources/0dhYG) course resource._

---

# Assignment 2 - Pandas Introduction
All questions are weighted the same in this assignment.
## Part 1
The following code loads the olympics dataset (olympics.csv), which was derrived from the Wikipedia entry on [All Time Olympic Games Medals](https://en.wikipedia.org/wiki/All-time_Olympic_Games_medal_table), and does some basic data cleaning. 

The columns are organized as # of Summer games, Summer medals, # of Winter games, Winter medals, total # number of games, total # of medals. Use this dataset to answer the questions below.

In [2]:
import pandas as pd

df = pd.read_csv('olympics.csv', index_col=0, skiprows=1)

for col in df.columns:
    if col[:2]=='01':
        df.rename(columns={col:'Gold'+col[4:]}, inplace=True)
    if col[:2]=='02':
        df.rename(columns={col:'Silver'+col[4:]}, inplace=True)
    if col[:2]=='03':
        df.rename(columns={col:'Bronze'+col[4:]}, inplace=True)
    if col[:1]=='№':
        df.rename(columns={col:'#'+col[1:]}, inplace=True)

names_ids = df.index.str.split('\s\(') # split the index by '('

df.index = names_ids.str[0] # the [0] element is the country name (new index) 
df['ID'] = names_ids.str[1].str[:3] # the [1] element is the abbreviation or ID (take first 3 characters from that)

df = df.drop('Totals')
df.head()

,# Summer,Gold,Silver,Bronze,Total,# Winter,Gold.1,Silver.1,Bronze.1,Total.1,# Games,Gold.2,Silver.2,Bronze.2,Combined total,ID
Afghanistan,13,0,0,2,2,0,0,0,0,0,13,0,0,2,2,AFG
Algeria,12,5,2,8,15,3,0,0,0,0,15,5,2,8,15,ALG
Argentina,23,18,24,28,70,18,0,0,0,0,41,18,24,28,70,ARG
Armenia,5,1,2,9,12,6,0,0,0,0,11,1,2,9,12,ARM
Australasia,2,3,4,5,12,0,0,0,0,0,2,3,4,5,12,ANZ


### Question 0 (Example)

What is the first country in df?

*This function should return a Series.*

In [2]:
# You should write your whole answer within the function provided. The autograder will call
# this function and compare the return value against the correct solution value
def answer_zero():
    # This function returns the row for Afghanistan, which is a Series object. The assignment
    # question description will tell you the general format the autograder is expecting
    return df.iloc[0]

# You can examine what your function returns by calling it in the cell. If you have questions
# about the assignment formats, check out the discussion forums for any FAQs
answer_zero() 

# Summer           13
Gold                0
Silver              0
Bronze              2
Total               2
# Winter            0
Gold.1              0
Silver.1            0
Bronze.1            0
Total.1             0
# Games            13
Gold.2              0
Silver.2            0
Bronze.2            2
Combined total      2
ID                AFG
Name: Afghanistan, dtype: object

### Question 1
Which country has won the most gold medals in summer games?

*This function should return a single string value.*

In [3]:
def answer_one():
    return df['Gold'].idxmax()


### Question 2
Which country had the biggest difference between their summer and winter gold medal counts?

*This function should return a single string value.*

In [4]:
def answer_two():
    df_copy = df.copy()
    df_copy['diff'] = abs(df['Gold'] - df['Gold.1'])
    return df_copy['diff'].idxmax()


### Question 3
Which country has the biggest difference between their summer gold medal counts and winter gold medal counts relative to their total gold medal count? 

$$\frac{Summer~Gold - Winter~Gold}{Total~Gold}$$

Only include countries that have won at least 1 gold in both summer and winter.

*This function should return a single string value.*

In [5]:
def answer_three():
    min1gold = df.where((df['Gold'] > 0) & (df['Gold.1'] > 0))
    min1gold = min1gold.dropna()
    min1gold['RelativeGold'] = ((df['Gold'] - df['Gold.1'])/df['Gold.2'])
    return min1gold['RelativeGold'].idxmax()
    return min1gold

### Question 4
Write a function that creates a Series called "Points" which is a weighted value where each gold medal (`Gold.2`) counts for 3 points, silver medals (`Silver.2`) for 2 points, and bronze medals (`Bronze.2`) for 1 point. The function should return only the column (a Series object) which you created.

*This function should return a Series named `Points` of length 146*

In [3]:
pd.set_option('display.max_rows', len(df))

def answer_four():
    points_df = df.copy()
    points_df['points'] = ((points_df['Gold.2'] * 3) + (points_df['Silver.2'] * 2) + (points_df['Bronze.2'] * 1))
    Points = pd.Series(points_df['points'].values)
    return Points

answer_four()

0         2
1        27
2       130
3        16
4        22
5       923
6       569
7        43
8        24
9         1
10        1
11      154
12      276
13        1
14        5
15        2
16      184
17        2
18      411
19        3
20       12
21      846
22       24
23     1120
24       29
25        7
26        2
27       67
28      420
29        2
30      134
31      327
32      335
33        1
34       14
35        5
36       49
37        1
38       77
39       94
40      895
41     1500
42        2
43       42
44     1546
45      269
46     1068
47      459
48        5
49     1574
50      213
51        3
52        2
53        1
54        3
55        6
56      962
57        6
58       50
59       49
60      110
61        1
62       55
63       10
64     1333
65      131
66      866
67      113
68      168
69       90
70      609
71        2
72        4
73       47
74        6
75       15
76       38
77        9
78        1
79        9
80        1
81      109
82        9
83  

## Part 2
For the next set of questions, we will be using census data from the [United States Census Bureau](http://www.census.gov/popest/data/counties/totals/2015/CO-EST2015-alldata.html). Counties are political and geographic subdivisions of states in the United States. This dataset contains population data for counties and states in the US from 2010 to 2015. [See this document](http://www.census.gov/popest/data/counties/totals/2015/files/CO-EST2015-alldata.pdf) for a description of the variable names.

The census dataset (census.csv) should be loaded as census_df. Answer questions using this as appropriate.

### Question 5
Which state has the most counties in it? (hint: consider the sumlevel key carefully! You'll need this for future questions too...)

*This function should return a single string value.*

In [101]:
census_df = pd.read_csv('census.csv')
census_df.head()

,SUMLEV,REGION,DIVISION,STATE,COUNTY,STNAME,CTYNAME,CENSUS2010POP,ESTIMATESBASE2010,POPESTIMATE2010,...,RDOMESTICMIG2011,RDOMESTICMIG2012,RDOMESTICMIG2013,RDOMESTICMIG2014,RDOMESTICMIG2015,RNETMIG2011,RNETMIG2012,RNETMIG2013,RNETMIG2014,RNETMIG2015
0,40,3,6,1,0,Alabama,Alabama,4779736,4780127,4785161,...,0.002295,-0.193196,0.381066,0.582002,-0.467369,1.030015,0.826644,1.383282,1.724718,0.712594
1,50,3,6,1,1,Alabama,Autauga County,54571,54571,54660,...,7.242091,-2.915927,-3.012349,2.265971,-2.530799,7.606016,-2.626146,-2.722002,2.592270,-2.187333
2,50,3,6,1,3,Alabama,Baldwin County,182265,182265,183193,...,14.832960,17.647293,21.845705,19.243287,17.197872,15.844176,18.559627,22.727626,20.317142,18.293499
3,50,3,6,1,5,Alabama,Barbour County,27457,27457,27341,...,-4.728132,-2.500690,-7.056824,-3.904217,-10.543299,-4.874741,-2.758113,-7.167664,-3.978583,-10.543299
4,50,3,6,1,7,Alabama,Bibb County,22915,22919,22861,...,-5.527043,-5.068871,-6.201001,-0.177537,0.177258,-5.088389,-4.363636,-5.403729,0.754533,1.107861


In [8]:
def answer_five():
    withinstatedf = census_df[census_df['SUMLEV'] == 50]
    grouped = withinstatedf.groupby(['STNAME']).count()
    return grouped['COUNTY'].idxmax()


### Question 6
Only looking at the three most populous counties for each state, what are the three most populous states (in order of highest population to lowest population)? Use `CENSUS2010POP`.

*This function should return a list of string values.*

In [88]:
def answer_six():
    #ctyleveldata = census_df[census_df['SUMLEV'] == 50]
    #columns_to_keep = ['STNAME',
    #               'CTYNAME',
    #               'CENSUS2010POP']
    #ctyleveldata = ctyleveldata[columns_to_keep]
    #grouped_ctypopdata = ctyleveldata.sort(['STNAME','CENSUS2010POP'],ascending=False)
    #return grouped_ctypopdata
    ctyleveldata = census_df[census_df['SUMLEV'] == 50]
    columns_to_keep = ['STNAME',
                   'CTYNAME',
                  'CENSUS2010POP']
    ctyleveldata = ctyleveldata[columns_to_keep]
    df_agg = ctyleveldata.groupby(['STNAME', 'CTYNAME']).agg({'CENSUS2010POP' : sum})
    #res = df_agg.sort(['CENSUS2010POP'], ascending=False)
    g = df_agg['CENSUS2010POP'].groupby(level=0, group_keys=False)
    res = g.apply(lambda x: x.order(ascending=False).head(3))
    top3ctys = g.nlargest(3)
    totalPop = {}
    for (key, value) in top3ctys.iteritems():
        ctyname = key[0]
        if ctyname in totalPop:
            totalPop[ctyname] += value
        else:
            totalPop[ctyname] = value
    finalList = sorted(totalPop.items(), key=lambda x: x[1], reverse=True)
    topPopulatedStates = []
    topPopulatedStates.append(finalList[0][0])
    topPopulatedStates.append(finalList[1][0])
    topPopulatedStates.append(finalList[2][0])
    return topPopulatedStates

### Question 7
Which county has had the largest absolute change in population within the period 2010-2015? (Hint: population values are stored in columns POPESTIMATE2010 through POPESTIMATE2015, you need to consider all six columns.)

e.g. If County Population in the 5 year period is 100, 120, 80, 105, 100, 130, then its largest change in the period would be |130-80| = 50.

*This function should return a single string value.*

In [10]:
def answer_seven():
    allcounties = census_df[census_df['SUMLEV'] == 50]
    columns_to_keep = ['STNAME', 'CTYNAME','POPESTIMATE2010',
                   'POPESTIMATE2011',
                   'POPESTIMATE2012',
                   'POPESTIMATE2013',
                   'POPESTIMATE2014',
                   'POPESTIMATE2015']
    allcty_popdata = allcounties[columns_to_keep]
    allcty_popdata_subset = allcty_popdata. set_index('CTYNAME')[['POPESTIMATE2010', 'POPESTIMATE2011', 
                                                                  'POPESTIMATE2012', 'POPESTIMATE2013',
                                                                 'POPESTIMATE2014', 'POPESTIMATE2015']]
    abschangedf = allcty_popdata_subset.max(axis=1) - allcty_popdata_subset.min(axis=1)
    cty_largepop = abschangedf[abschangedf == abschangedf.values.max()]
    return cty_largepop.index[0]


### Question 8
In this datafile, the United States is broken up into four regions using the "REGION" column. 

Create a query that finds the counties that belong to regions 1 or 2, whose name starts with 'Washington', and whose POPESTIMATE2015 was greater than their POPESTIMATE 2014.

*This function should return a 5x2 DataFrame with the columns = ['STNAME', 'CTYNAME'] and the same index ID as the census_df (sorted ascending by index).*

In [114]:
pd.set_option('display.max_rows', len(census_df))
def answer_eight():
    columns_to_keep = ['SUMLEV', 'REGION', 'STNAME', 'CTYNAME','POPESTIMATE2010',
                   'POPESTIMATE2011',
                   'POPESTIMATE2012',
                   'POPESTIMATE2013',
                   'POPESTIMATE2014',
                   'POPESTIMATE2015']
    first_df = census_df[columns_to_keep]
    region_one_two_df = first_df[((census_df['REGION'] == 1) | (census_df['REGION'] == 2)) 
                                  & (census_df['SUMLEV'] == 50)]
    region_washington_df = region_one_two_df.where(region_one_two_df['CTYNAME'].str.startswith('Washington')).dropna()
    estimated_df = region_washington_df.where(region_washington_df['POPESTIMATE2015'] > region_washington_df['POPESTIMATE2014'])
    estimated_valid_df = estimated_df.dropna()
    
    return estimated_valid_df[['STNAME', 'CTYNAME']]


,STNAME,CTYNAME
896,Iowa,Washington County
1419,Minnesota,Washington County
2345,Pennsylvania,Washington County
2355,Rhode Island,Washington County
3163,Wisconsin,Washington County
